# Optimization practical
Guido Raos, Politecnico di Milano, April 2021 (guido.raos@polimi.it)

In this notebook we test some optimization methods, which are used also in a molecular modeling contexts (e.g., to minimize the energy of a molecular system with respect to the atomic coordinates).

All the methods are implemented within the Optimize package of SciPy:
https://docs.scipy.org/doc/scipy/reference/optimize.html?highlight=optimiz#

We test the optimization methods on:

* The Himmelblau function, a standard two-dimensional function used for testing comparing optimization algorithms: https://en.wikipedia.org/wiki/Test_functions_for_optimization

* clusters of $N$ atoms interacting by the LJ potential:
\begin{equation*}
U_{LJ}(\mathbf{R}_0,...,\mathbf{R}_{N-1}) = \sum_{i=0}^{N-2} \sum_{j=i+1}^{N-1} 4\epsilon \left[ \left(\frac{\sigma}{R_{ij}}\right)^{12} - \left(\frac{\sigma}{R_{ij}}\right)^{6}\right]
\end{equation*}

In [ ]:
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt

## The Himmelblau function

In [ ]:

def himmel(x):
    """Returns the Himmelblau function"""
    return (x[0]**2+x[1]-11)**2 + (x[0]+x[1]**2)-7

def himmel_der(x):
    """Returns the gradient of the Himmelblau function"""
    g = np.zeros(2)
    g[0] = 4*x[0]*( x[0]**2 +x[1] -11 ) + 2*( x[0] +x[1]**2 -7 )
    g[1] = 2* ( x[0]**2 +x[1] -11) +4*x[1]*( x[0] +x[1]**2 -7)
    return g
    
def himmel_hess(x):
    """Returns the Hessian of the Himmelblau function"""
    h = np.zeros([2,2])
    h[0,0] = 12*x[0]**2 + 4*x[1] -42
    h[1,1] = 4*x[1] + 12*x[1] -26
    h[0,1] = h[1,0] = 4*x[0] + 4*x[1]
    return h
    

I wanted to do something similar to this ... 

https://scipy-lectures.org/advanced/mathematical_optimization/auto_examples/plot_gradient_descent.html

No time today, perhaps on another day...

## Lennard-Jones clusters: local optimization

In [ ]:
def ULJ(r, sigma=1.0, epsilon=1.0):
    """Computes the LJ interaction energy for two atoms, at distance r"""
#    print("Now in ULJ", r, sigma, epsilon)
    sr6 = (sigma / r)**6
    energy = 4 * epsilon * ( sr6**2 - sr6 )
    return energy

def ULJ_der(r, sigma=1.0, epsilon=1.0):
    """Computes the derivative of the LJ energy for two atoms, at distance r"""
    sr = sigma / r
    sr6 = sr**6
    sr7 = sr*sr6
    sr13 = sr6*sr7
    derivative = 24 * (epsilon/sigma) * ( -2.0*sr13 + sr7 )
    return derivative

def ULJmany(coords1D, sigma=1.0, epsilon=1.0):
    """Computes the LJ interaction energy for N atoms, with 3D coordinates x"""
    N = coords1D.shape[0]//3
#    print ("Number of atoms in LJ Energy:", N)
    x = coords1D.reshape(N,3,order='C')
    energy = 0.0
    for i in range(N-1):
        for j in range(i+1,N):
            rji = (x[i,0]-x[j,0])**2 + (x[i,1]-x[j,1])**2 + (x[i,2]-x[j,2])**2
            rji = np.sqrt(rji)
            energy += ULJ(rji, sigma, epsilon)
    return energy

def ULJmany_der(coords1D, sigma=1.0, epsilon=1.0):
    """Computes the gradient of the LJ energy for N atoms, with 3D coordinates x"""
    N = coords1D.shape[0]//3
#    print ("Number of atoms in LJ Gradient:", N)
    x = coords1D.reshape(N,3,order='C')
    gradient = np.zeros([N,3])
    for i in range(N-1):
        for j in range(i+1,N):
            xji = x[i,0]-x[j,0]
            yji = x[i,1]-x[j,1]
            zji = x[i,2]-x[j,2]
            rji = xji**2 + yji**2 + zji**2
            rji = np.sqrt(rji)
            gji = ULJ_der(rji, sigma, epsilon) / rji
            gradient[i,0] += gji * xji
            gradient[i,1] += gji * yji
            gradient[i,2] += gji * zji
            gradient[j,0] -= gji * xji
            gradient[j,1] -= gji * yji
            gradient[j,2] -= gji * zji        
    gradient1D = gradient.reshape(3*N, order='C')
    return gradient1D


In [ ]:
# A simple function for writing an xyz file
def printxyz(coords, filename="test.xyz", comment="Atomic coordinates", scale=1.0, mode="w"):
    N = (coords.shape)[0]
    print("Writing", N, "atomic coordinates to file", filename)
    f = open(filename,mode)
    f.write(str(N) + '\n')
    f.write(comment + '\n')
    for i in range(N):
        xyz = str(coords[i,0]*scale).rjust(16) +' '+ \
              str(coords[i,1]*scale).rjust(16) +' '+ \
              str(coords[i,2]*scale).rjust(16) 
        f.write('LJ ' + xyz + '\n')
    f.close()
    return


The starting geometry for the LJ clusters will be "blocks" with $N_1\times N_2\times N_3$ atoms on each side, at a distance $D$. We assume that, in the Lennard-Jones potential, $\sigma=1$ and $\epsilon=1$.

In [ ]:
# Define the starting geometry: a rectangular "box" with N1*N2*N3 atoms, spaced by D.
# We slightly scamble the positions of the atoms, in order to "break the symmetry" of the box.
# See the calls to no.random.normal(), below.
# We do it because the optimization tends to get "stuck" in highly symmetric configurations.
D=1.12
N1, N2, N3 = 13, 1, 1,

N = N1*N2*N3
coords = np.ndarray([N,3])
index = -1
for i in range(N1):
    x = i*D
    for j in range(N2):
        y = j*D
        for k in range(N3):
            z = k*D
            index +=1
            coords[index,0] = x + np.random.normal(loc=0.0, scale=0.05*D)
            coords[index,1] = y + np.random.normal(loc=0.0, scale=0.05*D)
            coords[index,2] = z + np.random.normal(loc=0.0, scale=0.05*D)
            
# Reshape the array, from a matrix [N,3] to a vector [3*N].
# (1D vector of coordinates is expected by ULJ and ULJ_der functions, and by SciPy's optimization methods).
coords1D = coords.reshape(3*N, order='C')

# Compute the energy 
energy0 = ULJmany(coords1D)
print(energy0)

# Write out the starting structure and energy to another xyz file.
comment = 'Starting LJ cluster with Energy '+str(energy0)
printxyz(coords, filename='cluster_0.xyz', comment=comment)

# Minimize the energy of the cluster
result = opt.minimize(ULJmany, coords1D, method='CG', jac=ULJmany_der, tol=1.0e-5, options={'maxiter': 200, 'disp': True} )

# Write out the optimized structure and energy to another xyz file.
finalcoords1D = result.x
finalenergy = result.fun
finalcoords = finalcoords1D.reshape(N, 3, order='C')
comment = 'Optimized LJ cluster with Energy '+str(finalenergy)
printxyz(finalcoords, filename='cluster_final.xyz', comment=comment)


# Lennard-Jones clusters: global optimization

Now let us attempt to find the absolute minumum for a cluster of $N$ Lennard-Jones atoms. This is a **global optimization** problem, similar to that of the [traveling salesman](https://en.wikipedia.org/wiki/Travelling_salesman_problem). Another optimization problem of this type is that of [protein folding](https://en.wikipedia.org/wiki/Protein_folding). These problems are notoriously hard.

We will use two methods implemented within [SciPy](https://docs.scipy.org/doc/scipy/reference/optimize.html?highlight=optimize#module-scipy.optimize):
* [Differential Evolution (DE)](https://en.wikipedia.org/wiki/Differential_evolution), a method originally proposed by [Storn and Price](https://link.springer.com/article/10.1023/A:1008202821328) in 1997. This falls within the broader class of [evolutionary algorithms](https://en.wikipedia.org/wiki/Evolutionary_algorithm). This method does not require the gradient of the objective function (the cluster's potential energy).
* [Basin Hopping (BH)](https://en.wikipedia.org/wiki/Basin-hopping), a method introduced by [Wales and Doye](https://pubs.acs.org/doi/10.1021/jp970984n), also in 1997. The method iterates by performing a random perturbation of the coordinates, performing a local optimization, and accepting or rejecting the new coordinates based on the minimized function value. The acceptance or rejection of a step is based on the Metropolis Monte Carlo algorithm, which depends on a fictitious temperature $T$ (more about this later, in this course). This method may or may not require derivative information in the local optimization steps, depending on the chosen minimization method (e.g., conjugate gradients or other).

The paper by Wales and Doye is especially interesting for us, since it is explicitly devoted to the global minimum of LJ clusters, from two up to $N$=110 atoms.
In the introduction of the paper, the authors point out that the number of local minima increases very quickly with $N$, and by extrapolation they estimate that this number could be as large as $10^{60}$ for $N=147$ (excluding the trivial $N!$ factor arising from permutations of the atoms). 
They also point out that the difficulty of the global optimization problem does not increase monotonically with the system size $N$. Some clusters are "easy" (e.g., for "magic numbers" $N$=13, 55, 147,...) because they have well-defined global minima with an [icosahedral](https://en.wikipedia.org/wiki/Regular_icosahedron) symmetry. Other cluster sizes are quite challenging (e.g., $N$=38,75,76,77,102,103,104) because their global minima have unusual structures, with a narrow "basin".

In [ ]:
# BASIN HOPPING (WALES AND DOYE).

# Attempt to find the abolute minimum for a cluster of N LJ atoms.
# We use "reduced units", so that in the LJ potential epsilon=1 and sigma=1.

N = 13               # No. of atoms
Ntry = N             # No. of random starting points
rho0 = 0.1           # Initial density of atoms
L = (N/rho0)**(1/3)  # Atoms are placed randomly inside a cube of size L.

# define here the minimizer that will be used for the local optimizations
minimizer_kwargs = {"method":'CG', "jac":ULJmany_der, "tol":1.0e-5,}

Energies = []
xyzfile = "cluster_"+str(N)+"_BH.xyz"
mode = "w"
for i in range(Ntry):
    print()
    print("BASIN HOPPING. Starting geometry no.", i)
    coords = L*np.random.random_sample((N,3)) - L/2
    xyz = coords.reshape(3*N, order='C')
    res = opt.basinhopping(ULJmany, xyz, niter=100, T=0.8, stepsize=0.5,
            minimizer_kwargs=minimizer_kwargs, interval=50, disp=True)
    xyz = res.x
    energy = res.fun
    print("Attempt no.:", i, "  Energy:", energy)
    Energies.append(energy)
    coords = xyz.reshape(N, 3, order='C')
    comment = 'Optimized LJ cluster with Energy '+str(energy)
    printxyz(coords, filename=xyzfile, comment=comment, mode=mode)
    mode = "a"
Energies.sort()
print("FINAL ENERGIES FROM B.H. ON LJ CLUSTERS, N=", N)
print(Energies)


In [ ]:
# DEFFERENTIAL EVOLUTION (STORN AND PRICE).

# Attempt to find the abolute minimum for a cluster of N LJ atoms.
# We use "reduced units", so that in the LJ potential epsilon=1 and sigma=1.

N = 13               # No. of atoms
Ntry = N             # No. of random starting points
rho0 = 0.1           # Initial number density of atoms
L = (N/rho0)**(1/3)  # Atoms are placed randomly inside a cube of size L.

bounds = 3*N*[(-L/2,+L/2)]  #  Bounds on the atom coordinates

Energies = []
xyzfile = "cluster_"+str(N)+"_DE.xyz"
mode = "w"
for i in range(Ntry):
    print()
    print("DIFFERENTIAL EVOLUTION. Starting geometry no.", i)
    res = opt.differential_evolution(ULJmany, bounds=bounds, strategy='best1bin',
                maxiter=1000, popsize=15, tol=0.01, mutation=(0.5,1), recombination=0.7,
                disp=False, polish=True, init='random')
    xyz = res.x
    energy = res.fun
    print("Attempt no.:", i, "  Energy:", energy)
    Energies.append(energy)
    coords = xyz.reshape(N, 3, order='C')
    comment = 'Optimized LJ cluster with Energy '+str(energy)
    printxyz(coords, filename=xyzfile, comment=comment, mode=mode)  
    mode = "a"
Energies.sort()
print("FINAL ENERGIES FROM D.E. ON LJ CLUSTERS, N=", N)
print(Energies)
  

# Suggestions for further work

Each of the following could evolve into a end-of-course project, to be discussed at the exam:
* Perform a systematic study of the minima of LJ clusters for several $N$ values. Compare the results from different optimization algorithms. Compare the effect of changing some options within the global optimization algorithms. 
* Generalize the function for the LJ potential energy to the case where the atoms may be of different types (e.g., Ne and Xe), and characterize the resulting minima of the PES.
* Extend the study to clusters described by different interaction potentials (e.g., Buckingham or Morse).
* Etc. etc.